In [ ]:
# [1] - Célula de Inicialização
# Este código prepara o ambiente criando a estrutura principal para armazenar as dívidas.

# Este é o nosso caderno, onde vamos guardar as dívidas.
# Por enquanto, é uma lista vazia.
caderno_dividas = []

# Vamos imprimir para ver como ele está:
print(">>> Nosso caderno de dívidas começou assim:")
print(caderno_dividas)

In [ ]:
# [2] - Instalação de Bibliotecas Necessárias
# Instala a biblioteca oficial do Google para interagir com os modelos Gemini.
!pip install google-generativeai python-dotenv -q

In [ ]:
# [3] - Configuração do Modelo de Inteligência Artificial (Gemini)
# Importa as bibliotecas necessárias
import google.generativeai as genai
from google.colab import userdata # Importa o user data para pegar secrets

API_KEY = userdata.get('GEMINI_API_KEY')

# Configura a chave na biblioteca
genai.configure(api_key=API_KEY)

# Inicializa o modelo que vamos usar para interpretar texto
model = genai.GenerativeModel('gemini-2.0-flash')

print(">>> Modelo Gemini configurado e pronto para uso!")

In [ ]:
# --- Célula de Definição de Funções ---
# Este código define todas as funções usadas no Caderno de Dívidas.
# Rode esta célula para que as funções estejam prontas para serem usadas pelo loop principal.

# --- Função para Mostrar Resumo de Dívidas ---
def mostrar_resumo_dividas(lista_de_dividas):
    """
    Calcula e exibe o total de dívidas PENDENTES para cada pessoa.
    """
    total_por_pessoa = {}

    # Percorre cada dívida para somar os totais pendentes por pessoa.
    for divida in lista_de_dividas:
        # Verifica se a dívida não está marcada como paga.
        if not divida.get('paga', False):
            pessoa = divida.get('pessoa')
            valor = divida.get('valor')

            if pessoa and valor is not None:
                # Adiciona o valor ao total da pessoa. Se a pessoa ainda não estiver no dicionário, começa com o valor dela.
                total_por_pessoa[pessoa] = total_por_pessoa.get(pessoa, 0) + valor

    print("\n>>> Resumo de Dívidas Pendentes:")
    # Verifica se há alguma dívida pendente para mostrar.
    if not total_por_pessoa:
        print("Nenhuma dívida pendente encontrada.")
    else:
        # Exibe o total pendente para cada pessoa.
        for pessoa, total in total_por_pessoa.items():
            valor_formatado = f"R$ {total:.2f}".replace('.', ',')
            print(f"- {pessoa}: {valor_formatado}")

    print("--------------------------")


# --- Função para Adicionar Nova Dívida ---
def adicionar_divida(lista_de_dividas, pessoa, valor, motivo=""):
    """
    Adiciona uma nova anotação de dívida à lista principal.
    Motivo é opcional, padrão é vazio.
    """
    # Cria um dicionário para representar a nova dívida.
    nova_divida = {
        'pessoa': pessoa,
        'valor': float(valor), # Garante que o valor é um número.
        'motivo': motivo,
        'paga': False # Nova dívida começa sempre como não paga.
    }
    # Adiciona o dicionário da nova dívida à lista principal.
    lista_de_dividas.append(nova_divida)
    print(f"Dívida de R$ {float(valor):.2f} anotada para {pessoa}.") # Confirmação para o usuário.


# --- Função para Interpretar Comando com IA (Gemini) ---
def interpretar_comando_ia(frase_usuario):
    """
    Usa o modelo Gemini para interpretar a frase do usuário, extraindo intenção e dados.
    O prompt guia o modelo sobre quais intenções e entidades procurar.
    Retorna um dicionário com 'intent' e dados extraídos.
    """
    # O texto grande do prompt que instrui o modelo Gemini.
    # Este prompt foi refinado em passos anteriores para entender as intenções e extrair dados.
    prompt = f"""
Você é um assistente de um caderno de dívidas que anota o que as pessoas devem ao proprietário.
Sua tarefa é interpretar a frase do usuário e extrair a intenção principal e as informações relevantes (entidades).

As intenções possíveis que você deve identificar são:
- 'adicionar_divida': O usuário quer anotar que alguém AGORA deve um valor novo.
- 'mostrar_resumo': O usuário quer ver as dívidas totais pendentes por pessoa.
- 'marcar_pago': O usuário está registrando um pagamento recebido ou quer marcar uma dívida (total ou parcial) como paga. Você DEVE extrair o valor pago se ele for mencionado.
- 'ajuda': O usuário pede ajuda ou informações sobre como usar o caderno.
- 'sair': O usuário quer encerrar o programa.
- 'ver_individuais': O usuário quer ver a lista detalhada de dívidas de uma pessoa.
- 'desconhecido': A intenção não se encaixa em nenhuma das anteriores.

Para as intenções 'adicionar_divida' e 'marcar_pago', extraia as seguintes informações (entidades):
- pessoa: O nome da pessoa envolvida na transação.
- valor: O valor numérico. Para 'adicionar_divida', é o valor devido. Para 'marcar_pago', é o valor pago. Use ponto como separador decimal.
- motivo: Uma breve descrição do motivo (para 'adicionar_divida') ou referência ao pagamento (para 'marcar_pago', ex: "pagamento parcial", "quitação").

Formato de saída:
Responda APENAS em formato JSON.
O JSON deve conter as chaves "intent" e "dados".
O objeto "dados" deve ser um dicionário contendo as entidades extraídas (pessoa, valor, motivo) se forem relevantes e encontradas para a intenção. Se uma informação não for encontrada, use null ou vazio apropriado. O valor deve ser numérico (float).

Exemplos de entrada e saída (para ajudar o modelo a aprender o formato):

Entrada: "Quero anotar que o João me deve 50 reais pela pizza"
Saída: {{"intent": "adicionar_divida", "dados": {{"pessoa": "João", "valor": 50.0, "motivo": "pizza"}}}}

Entrada: "Anotar 120 da Maria, conserto do carro"
Saída: {{"intent": "adicionar_divida", "dados": {{"pessoa": "Maria", "valor": 120.0, "motivo": "conserto do carro"}}}}

Entrada: "o Pedro pegou 30 conto"
Saída: {{"intent": "adicionar_divida", "dados": {{"pessoa": "Pedro", "valor": 30.0, "motivo": null}}}}

Entrada: "quanto o Pedro deve no total?"
Saída: {{"intent": "mostrar_resumo", "dados": {{}}}}

Entrada: "Vê aí as dívidas da Ana"
Saída: {{"intent": "mostrar_resumo", "dados": {{"pessoa": "Ana"}}}}

Entrada: "Marcar como pago as dívidas do Bruno"
Saída: {{"intent": "marcar_pago", "dados": {{"pessoa": "Bruno", "valor": null, "motivo": "quitação total"}}}}

Entrada: "Pedro me pagou 5 reais"
Saída: {{"intent": "marcar_pago", "dados": {{"pessoa": "Pedro", "valor": 5.0, "motivo": "pagamento parcial"}}}}

Entrada: "Tira 25 reais da conta do João"
Saída: {{"intent": "marcar_pago", "dados": {{"pessoa": "João", "valor": 25.0, "motivo": "pagamento"}}}}

Entrada: "Zerar dívida da Maria"
Saída: {{"intent": "marcar_pago", "dados": {{"pessoa": "Maria", "valor": null, "motivo": "quitação total"}}}}

Entrada: "Ver dívidas do Joao"
Saída: {{"intent": "ver_individuais", "dados": {{"pessoa": "Joao"}}}}

Entrada: "Quais as dívidas da Maria?"
Saída: {{"intent": "ver_individuais", "dados": {{"pessoa": "Maria"}}}}

Entrada: "Detalhes do que o Pedro deve"
Saída: {{"intent": "ver_individuais", "dados": {{"pessoa": "Pedro"}}}}

Entrada: "sair do programa"
Saída: {{"intent": "sair", "dados": {{}}}}

Entrada: "o que você faz?"
Saída: {{"intent": "ajuda", "dados": {{}}}}

Entrada: "{frase_usuario}"

""" # Fim do prompt da IA

    try:
        # Chama o modelo Gemini para gerar conteúdo baseado no prompt.
        response = model.generate_content(prompt)

        # Processa a resposta do modelo, esperando um formato JSON.
        import json
        # Tenta limpar a resposta para garantir que é um JSON válido.
        response_text = response.text.strip().replace('```json', '').replace('```', '').strip()

        # Converte a string JSON para um dicionário Python.
        resultado_ia = json.loads(response_text)

        # Extrai a intenção e os dados do resultado da IA.
        intent = resultado_ia.get('intent', 'desconhecido')
        dados_extraidos = resultado_ia.get('dados', {})

        # Trata o valor extraído, garantindo que é um float se existir.
        if 'valor' in dados_extraidos and dados_extraidos['valor'] is not None:
             try:
                 dados_extraidos['valor'] = float(dados_extraidos['valor'])
             except (ValueError, TypeError):
                 dados_extraidos['valor'] = None # Marca como None se a conversão falhar.

    except Exception as e:
        # Em caso de qualquer erro (API, JSON inválido, etc.), define a intenção como desconhecida.
        print(f"Erro ao comunicar com o modelo IA ou processar resposta: {e}")
        # print("Resposta bruta recebida (se houver):", response.text if 'response' in locals() else "Nenhuma") # Opcional para debug
        intent = "desconhecido"
        dados_extraidos = {}

    return {'intent': intent, 'dados': dados_extraidos}


# --- Função para Marcar Dívidas como Pagas ou Registrar Pagamento Parcial ---
def marcar_dividas_pagas(lista_de_dividas, pessoa, valor_pago=None):
    """
    Registra pagamento para uma pessoa.
    Se valor_pago for None ou <= 0, marca TODAS as dívidas pendentes para a pessoa como pagas.
    Se valor_pago for > 0, aplica o pagamento contra as dívidas pendentes (mais antigas primeiro),
    reduzindo os valores ou marcando-as como pagas até o valor_pago ser consumido.
    """
    # Lógica para marcar todas as dívidas pendentes (quando não há valor de pagamento especificado ou é <= 0).
    if valor_pago is None or valor_pago <= 0:
        contador_marcadas_total = 0
        indices_dividas_pendentes = []
        for i, divida in enumerate(lista_de_dividas):
             if divida.get('pessoa') == pessoa and not divida.get('paga', False):
                  indices_dividas_pendentes.append(i)

        if not indices_dividas_pendentes:
             print(f"Nenhuma dívida pendente encontrada para {pessoa}.")
             return

        print(f"Marcando TODAS as {len(indices_dividas_pendentes)} dívida(s) pendente(s) de {pessoa} como paga(s) (Sem valor de pagamento especificado ou valor <= 0).")

        for index in indices_dividas_pendentes:
             lista_de_dividas[index]['paga'] = True
             contador_marcadas_total += 1

        return

    # Lógica para aplicar pagamento parcial/total com valor especificado.
    print(f"Aplicando pagamento de R$ {valor_pago:.2f} para {pessoa}...")
    valor_restante_pagamento = valor_pago
    dividas_afetadas_count = 0
    valor_aplicado_total = 0

    # Coleta informações das dívidas pendentes para facilitar a aplicação do pagamento.
    pending_debts_info = []
    for i, divida in enumerate(lista_de_dividas):
        if divida.get('pessoa') == pessoa and not divida.get('paga', False):
            pending_debts_info.append({'index': i, 'divida': divida, 'valor_original': divida.get('valor_original', divida['valor'])})

    if not pending_debts_info:
         print(f"Nenhuma dívida pendente encontrada para {pessoa} para aplicar o pagamento.")
         if valor_restante_pagamento > 0:
             print(f"Restou R$ {valor_restante_pagamento:.2f} do pagamento que não foi aplicado.")
         return

    # Itera sobre as dívidas pendentes coletadas e aplica o pagamento.
    for debt_info in pending_debts_info:
        original_index = debt_info['index']
        divida_referencia = lista_de_dividas[original_index]

        valor_pendente_nesta_divida = divida_referencia['valor']

        # Verifica se o pagamento restante cobre esta dívida.
        if valor_restante_pagamento >= valor_pendente_nesta_divida:
            print(f"  - Pagamento de R$ {valor_pendente_nesta_divida:.2f} cobre a dívida de R$ {valor_pendente_nesta_divida:.2f}.")
            valor_restante_pagamento -= valor_pendente_nesta_divida
            divida_referencia['paga'] = True
            divida_referencia['valor'] = 0
            dividas_afetadas_count += 1
            valor_aplicado_total += valor_pendente_nesta_divida

        # Se o pagamento restante não cobre esta dívida inteira.
        else:
            print(f"  - Pagamento de R$ {valor_restante_pagamento:.2f} cobre parcialmente a dívida de R$ {valor_pendente_nesta_divida:.2f}.")
            divida_referencia['valor'] -= valor_restante_pagamento
            valor_aplicado_total += valor_restante_pagamento
            valor_restante_pagamento = 0
            dividas_afetadas_count += 1

        # Para o loop se o valor de pagamento foi todo usado.
        if valor_restante_pagamento <= 0:
            break

    # Relata o resultado da aplicação do pagamento.
    if dividas_afetadas_count > 0:
         print(f"Total de {dividas_afetadas_count} dívida(s) de {pessoa} afetada(s) pelo pagamento de R$ {valor_aplicado_total:.2f}.")
    # else: # Esta mensagem só seria atingida em casos específicos onde o valor_pago era <=0, tratado no início.
         # print(f"Nenhuma dívida pendente encontrada para {pessoa} para aplicar o pagamento (caminho alternativo).")


    if valor_restante_pagamento > 0:
        print(f"Restou R$ {valor_restante_pagamento:.2f} do pagamento que não foi aplicado.")


# --- Função para Mostrar Dívidas Individuais ---
def mostrar_dividas_individuais(lista_de_dividas, pessoa):
    """
    Mostra todas as dívidas (pendentes ou pagas) de uma pessoa específica, com detalhes.
    """
    print(f"\n>>> Dívidas de {pessoa}:")
    encontrou_dividas = False

    # Percorre cada dívida na lista para encontrar as da pessoa especificada.
    for i, divida in enumerate(lista_de_dividas):
        if divida.get('pessoa') == pessoa:
            encontrou_dividas = True
            valor_formatado = f"R$ {divida['valor']:.2f}".replace('.', ',')
            status = "Paga" if divida.get('paga', False) else "Pendente"
            motivo = divida.get('motivo', 'Sem motivo')
            # Imprime os detalhes de cada dívida encontrada.
            print(f"  - Dívida {i+1}: Valor: {valor_formatado}, Motivo: {motivo}, Status: {status}")

    # Mensagem se nenhuma dívida for encontrada para a pessoa.
    if not encontrou_dividas:
        print(f"Nenhuma dívida encontrada para {pessoa}.")

    print("--------------------------")

In [ ]:
# [9] - Célula do Loop Principal
# Este é o código que mantém o programa rodando, esperando por comandos do usuário,
# interpretando-os com a IA e executando as ações correspondentes.
# Rode esta célula para iniciar o chatbot interativo.

# Mensagens de boas-vindas e instruções iniciais para o usuário.
print("--- Olá! Sou seu Caderno de Dívidas Virtual (com IA!) ---")
print("--- Tente digitar a frase completa (ex: 'Quero anotar que o João me deve 50 reais') ---")
print("--- Digite 'ajuda' para ver os comandos básicos ou 'sair' para fechar ---")

# O loop principal do programa. 'while True' faz com que ele rode indefinidamente até ser interrompido.
while True:
    # Pede uma anotação ou pergunta ao usuário e remove espaços extras.
    frase_usuario = input(">>> Sua anotação ou pergunta: ").strip()

    # --- Verifica comandos diretos que não precisam de interpretação da IA ---
    # Comandos simples como 'sair' e 'ajuda' são tratados primeiro para resposta rápida.
    if frase_usuario.lower() == "sair":
        print("Fechando o caderno. Até mais!")
        # Em um projeto completo, salvaríamos os dados aqui antes de sair!
        break # Interrompe o loop 'while True', encerrando o programa.

    elif frase_usuario.lower() == "ajuda":
         print("\n--- Comandos disponíveis ---")
         print("  Tente digitar a frase completa sobre a dívida (ex: 'Anotar 100 do Pedro').")
         print("  Ou digite: 'mostrar resumo' para ver os totais.")
         print("  marcar pago: Para marcar as dívidas de uma pessoa como pagas. (Ex: 'Marcar pagas as dívidas do João')")
         print("  ver individual: Para ver a lista detalhada de dívidas de uma pessoa. (Ex: 'Ver dívidas do Pedro')")
         print("  Digite 'sair' para fechar o programa.")
         print("----------------------------")

    # --- Se não foi um comando direto, chama a função para interpretar a frase com IA ---
    else:
        # Chama a função que usa o modelo Gemini para entender a intenção e extrair dados da frase.
        interpretacao = interpretar_comando_ia(frase_usuario)
        # Extrai a intenção detectada (padrão 'desconhecido' se não encontrada).
        intent = interpretacao.get('intent', 'desconhecido')
        # Extrai os dados relevantes (pessoa, valor, motivo) da interpretação da IA.
        dados_extraidos = interpretacao.get('dados', {})

        # Linha opcional para depuração: mostra o que a IA detectou. Descomente se precisar ver.
        # print(f"(Debug: Intenção detectada: '{intent}', Dados: {dados_extraidos})")

        # --- Decide a ação a tomar com base na INTENÇÃO detectada pela IA ---

        # Se a intenção é mostrar o resumo das dívidas pendentes.
        if intent == "mostrar_resumo":
            mostrar_resumo_dividas(caderno_dividas)
            print("--------------------------")

        # Se a intenção é adicionar uma nova dívida.
        elif intent == "adicionar_divida":
            print("\n--- Tentando Anotar Nova Dívida (baseado na sua frase) ---")
            # Pega os dados extraídos pela IA.
            pessoa = dados_extraidos.get('pessoa')
            valor = dados_extraidos.get('valor')
            motivo = dados_extraidos.get('motivo', frase_usuario)

            # Verifica se a IA conseguiu extrair as informações essenciais para adicionar a dívida.
            if pessoa is not None and pessoa != '' and valor is not None and valor > 0:
                 try:
                     # Chama a função para adicionar a dívida.
                     adicionar_divida(caderno_dividas, pessoa, valor, motivo)
                 except Exception as e:
                     print(f"Ocorreu um erro ao adicionar a dívida extraída: {e}")
                     print("--------------------------")
            else:
                print("Não consegui extrair o nome e o valor da sua frase para anotar a dívida.")
                print("Tente digitar a frase de forma mais direta, por favor.")
                print("--------------------------")

        # Se a intenção é registrar um pagamento ou marcar dívidas como pagas.
        elif intent == "marcar_pago":
            print("\n--- Tentando Registrar Pagamento / Marcar Dívidas Pagas (baseado na sua frase) ---")
            # Pega os dados (pessoa e valor pago) extraídos pela IA.
            pessoa = dados_extraidos.get('pessoa')
            valor_pago = dados_extraidos.get('valor')

            # Verifica se a IA conseguiu extrair um nome de pessoa.
            if pessoa is not None and pessoa != '':
                # Chama a função que lida com pagamentos. Ela usará o valor_pago se for fornecido.
                marcar_dividas_pagas(caderno_dividas, pessoa, valor_pago=valor_pago)
                print("--------------------------")
            else:
                print("Não consegui identificar de quem você quer registrar o pagamento/marcar como pago.")
                print("Tente de outra forma, especificando o nome da pessoa na frase.")
                print("--------------------------")

        # Se a intenção é ver a lista detalhada de dívidas de uma pessoa específica.
        elif intent == "ver_individuais":
            print("\n--- Tentando Mostrar Dívidas Individuais (baseado na sua frase) ---")
            # Pega o nome da pessoa extraído pela IA.
            pessoa = dados_extraidos.get('pessoa')

            # Verifica se a IA conseguiu extrair um nome de pessoa.
            if pessoa is not None and pessoa != '':
                # Chama a função para mostrar as dívidas individuais.
                mostrar_dividas_individuais(caderno_dividas, pessoa)
            else:
                print("Não consegui identificar de quem você quer ver as dívidas.")
                print("Tente de outra forma, especificando o nome da pessoa na frase.")
                print("--------------------------")

        # Se a intenção detectada pela IA não corresponde a nenhuma ação conhecida.
        elif intent == "desconhecido":
            print("Não consegui entender o que você quer fazer com essa frase.")
            print("Digite 'ajuda' para ver os comandos ou tente de outra forma.")
            print("--------------------------")

# O loop termina quando a condição 'while True' se torna falsa (o que acontece apenas com o 'break' no comando 'sair').
# Se o programa sair do loop, a execução desta célula termina.

# --- Fim do Loop Principal ---